In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal

In [3]:
import koma.oma

In [4]:
k = 8000
m = 500
levels = 3
fs = 3.0
Tmax = 3600

In [5]:
# Create a shear frame
sf = strid.utils.ShearFrame(levels, k, m)
sf.set_rayleigh_damping_matrix([sf.get_natural_frequency(1), sf.get_natural_frequency(sf.n)], [.05]*2)

# Determine the time discretization and period
#Tmax = 1. / strid.w2f(sf.get_natural_frequency(1))
#fmax = strid.w2f(sf.get_natural_frequency(sf.n))
T = Tmax
#fs = 5 * fmax
t = np.arange(0., T, 1/fs)

# Define loads on system
## Unmeasureable: Stochastic loads on all floors (Process noise)
w = np.random.normal(size=(sf.n, t.size)) * 1e-1

## Load matrix, f
F = w.copy()

# Simulate response, accelerations at each floor measured
y0, _, _ = sf.simulate(t, F)

noise_factor = 1.0

noise = np.std(y0) * noise_factor

# Add measurement noise

y = y0 + noise

In [6]:
true_frequencies = np.array([sf.get_natural_frequency(i)/(2*np.pi) for i in range(1, sf.n+1)])
true_damping = np.array([sf.get_rayleigh_damping_ratio(i) for i in range(1, sf.n+1)])
true_modeshapes = np.array([sf.get_mode_shape(i) for i in range(1, sf.n+1)])

In [7]:
i = 30
s = 4

orders = np.arange(2, 60+2, 2)
stabcrit = {'freq':0.2, 'damping': 0.2, 'mac': 0.3}

In [8]:
lambd, phi = koma.oma.covssi(y, fs, i, orders) # find poles by Cov-SSI
lambd_stab, phi_stab, orders_stab, ix_stab = koma.oma.find_stable_poles(lambd, phi, orders, s, stabcrit=stabcrit, indicator='mac') # Stable poles

*** Covariance-driven SSI algorithm for OMA ***
> Establishing Hankel/Toeplitz matrices
  >> Correlation estimation
Maximum sample lag is larger than total sample length! Reducing to correspond to one below sample length.


KeyboardInterrupt: 